In [ ]:
import pandas as pd
path='/home/recordk/reco/BX-Book-Ratings.csv'
ratings = pd.read_csv(path)
ratings['Book-Rating'] = ratings['Book-Rating'].astype(int)
ratings.columns=['user_id','isbn','rating']
ratings=ratings[ratings['rating']!=0]
ratings=ratings.reset_index(drop=True)
from sklearn.preprocessing import LabelEncoder

# LabelEncoder
user_encoder = LabelEncoder()
isbn_encoder = LabelEncoder()

# user_id와 isbn label인코딩
ratings['user_id'] = user_encoder.fit_transform(ratings['user_id'])
ratings['isbn'] = isbn_encoder.fit_transform(ratings['isbn'])

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import numpy as np
# Variable 초기화 
K = 3                             # Latent factor 수 
lr = 0.01                           # 학습률
reg = 0.0001                     # Regularization penalty

def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# 모델 구조
def architecture(train_data, test_data,lr):
    # Keras model
    user = Input(shape=(1,))                                                # User input
    item = Input(shape=(1,))                                                # Item input
    P_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(user)     # (N, 1, K)
    Q_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(item)     # (M, 1, K)

    user_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(user)       # User bias term (N, 1, 1)
    item_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(item)       # Item bias term (M, 1, 1)

    #R = Dot(axes=2)([P_embedding, Q_embedding])                            # (N, 1, 1)
    R = layers.dot([P_embedding, Q_embedding], axes=2)
    #R = Add()([R, user_bias, item_bias])
    R = layers.add([R, user_bias, item_bias])
    R = Flatten()(R)                                                        # (N, 1)

    # Model setting
    model = Model(inputs=[user, item], outputs=R)
    model.compile(
    loss=RMSE,
    optimizer=SGD(lr=lr, momentum=0.9),
    # optimizer=Adamax(lr=0.005),
    metrics=[RMSE],
    )
    model.summary()
    size=len(ratings)
    # Model fitting
    result = model.fit(
    x=[train_data.user_id.values, train_data.isbn.values],
    y=train_data.rating.values - mu,
    epochs=85,
    batch_size=256,
    validation_data=(
        [test_data.user_id.values, test_data.isbn.values],
        test_data.rating.values - mu
    )
    )
    return model, result

# 사용자 수 및 도서 수
N = len(set(ratings.user_id)) + 1
M = len(set(ratings.isbn)) + 1


# kfold이용해서 트레인 테스트 3개로 분리
kf = KFold(n_splits=3, shuffle=True)

# rmse리스트
rmse_values = []

# for문
for train_indices, test_indices in kf.split(ratings):
    train_data = ratings.iloc[train_indices].reset_index(drop=True)
    test_data = ratings.iloc[test_indices].reset_index(drop=True)
    
    mu = train_data.rating.mean()    # 전체 평균 

    # 셔플
    train_data = shuffle(train_data)

    # 모델 및 결과
    model, result = architecture(train_data, test_data,lr)

    # 모델 평가
    predictions = model.predict([test_data.user_id.values, test_data.isbn.values]) + mu
    rmse = RMSE(test_data.rating.values, predictions)
    rmse_values.append(rmse)

# rmse 평균
average_rmse = np.mean(rmse_values)
print("평균 RMSE:", average_rmse)